In [ ]:
# Install Qiskit and PySCF
!pip install qiskit qiskit-nature pyscf
!pip install basis-set-exchange

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 22.5 MB/s eta 0:00:00


In [ ]:
#Comparison in sto-3g basis

# Upload .xyz file
from google.colab import files
uploaded = files.upload()

# Read the .xyz file and extract atomic coordinates
xyz_file = list(uploaded.keys())[0]
with open(xyz_file, 'r') as f:
    lines = f.readlines()
    atoms = lines[2:]  # Skip the first two lines (number of atoms and comment)
    atom_str = "; ".join(line.strip() for line in atoms)

print(f"Loaded molecule:\n{atom_str}")

# Import necessary libraries
from qiskit.circuit.library import EfficientSU2, RealAmplitudes, TwoLocal, PauliTwoDesign, ExcitationPreserving
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import VQE
from pyscf import gto, scf
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper

# Define function to run VQE with different ansatzes
def run_vqe_with_ansatz(atom_str, ansatz_type):
    # Setup molecule with PySCF
    mol = gto.M(atom=atom_str, basis="sto3g")
    h_f = scf.RHF(mol).run()

    # Use PySCFDriver to get the Hamiltonian
    driver = PySCFDriver(atom=atom_str)
    problem = driver.run()
    hamiltonian = problem.hamiltonian.second_q_op()

    # Map the Hamiltonian
    mapper = ParityMapper(num_particles=problem.num_particles)
    reduced_op = mapper.map(hamiltonian)

    # Setup ansatz
    if ansatz_type == 'EfficientSU2':
        ansatz = EfficientSU2(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'RealAmplitudes':
        ansatz = RealAmplitudes(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'TwoLocal':
        ansatz = TwoLocal(reduced_op.num_qubits, 'ry', 'cx', reps=1)
    elif ansatz_type == 'PauliTwoDesign':
        ansatz = PauliTwoDesign(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'ExcitationPreserving':
        ansatz = ExcitationPreserving(reduced_op.num_qubits, reps=1)
    else:
        raise ValueError(f"Unknown ansatz type: {ansatz_type}")

    # Setup VQE
    estimator = Estimator()
    optimizer = COBYLA()
    vqe = VQE(estimator, ansatz, optimizer)

    # Compute minimum eigenvalue
    result = vqe.compute_minimum_eigenvalue(reduced_op)

    return result, h_f.e_tot

# Run VQE with different ansatzes
ansatzes = ['EfficientSU2', 'RealAmplitudes', 'TwoLocal', 'PauliTwoDesign', 'ExcitationPreserving']
results = {}
for ansatz in ansatzes:
    vqe_result, scf_energy = run_vqe_with_ansatz(atom_str, ansatz)
    results[ansatz] = {
        'vqe_result': vqe_result,
        'scf_energy': scf_energy
    }

# Compare results
for ansatz, res in results.items():
    print(f"Ansatz: {ansatz}")
    print(f"VQE Eigenvalue: {res['vqe_result'].eigenvalue}")
    print(f"SCF Energy: {res['scf_energy']}")
    print(f"Optimal Parameters: {res['vqe_result'].optimal_parameters}")
    print(f"Optimizer Evaluations: {res['vqe_result'].optimizer_evals}")
    print(f"Optimizer Time: {res['vqe_result'].optimizer_time}")
    print("\n")


KeyboardInterrupt: 

In [ ]:
#Comparison in sto-6g basis

# Upload .xyz file
from google.colab import files
uploaded = files.upload()

# Read the .xyz file and extract atomic coordinates
xyz_file = list(uploaded.keys())[0]
with open(xyz_file, 'r') as f:
    lines = f.readlines()
    atoms = lines[2:]  # Skip the first two lines (number of atoms and comment)
    atom_str = "; ".join(line.strip() for line in atoms)

print(f"Loaded molecule:\n{atom_str}")

# Import necessary libraries
from qiskit.circuit.library import EfficientSU2, RealAmplitudes, TwoLocal, PauliTwoDesign, ExcitationPreserving
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import VQE
from pyscf import gto, scf
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper

# Define function to run VQE with different ansatzes
def run_vqe_with_ansatz(atom_str, ansatz_type):
    # Setup molecule with PySCF
    mol = gto.M(atom=atom_str, basis="sto6g")
    h_f = scf.RHF(mol).run()

    # Use PySCFDriver to get the Hamiltonian
    driver = PySCFDriver(atom=atom_str)
    problem = driver.run()
    hamiltonian = problem.hamiltonian.second_q_op()

    # Map the Hamiltonian
    mapper = ParityMapper(num_particles=problem.num_particles)
    reduced_op = mapper.map(hamiltonian)

    # Setup ansatz
    if ansatz_type == 'EfficientSU2':
        ansatz = EfficientSU2(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'RealAmplitudes':
        ansatz = RealAmplitudes(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'TwoLocal':
        ansatz = TwoLocal(reduced_op.num_qubits, 'ry', 'cx', reps=1)
    elif ansatz_type == 'PauliTwoDesign':
        ansatz = PauliTwoDesign(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'ExcitationPreserving':
        ansatz = ExcitationPreserving(reduced_op.num_qubits, reps=1)
    else:
        raise ValueError(f"Unknown ansatz type: {ansatz_type}")

    # Setup VQE
    estimator = Estimator()
    optimizer = COBYLA()
    vqe = VQE(estimator, ansatz, optimizer)

    # Compute minimum eigenvalue
    result = vqe.compute_minimum_eigenvalue(reduced_op)

    return result, h_f.e_tot

# Run VQE with different ansatzes
ansatzes = ['EfficientSU2', 'RealAmplitudes', 'TwoLocal', 'PauliTwoDesign', 'ExcitationPreserving']
results = {}
for ansatz in ansatzes:
    vqe_result, scf_energy = run_vqe_with_ansatz(atom_str, ansatz)
    results[ansatz] = {
        'vqe_result': vqe_result,
        'scf_energy': scf_energy
    }

# Compare results
for ansatz, res in results.items():
    print(f"Ansatz: {ansatz}")
    print(f"VQE Eigenvalue: {res['vqe_result'].eigenvalue}")
    print(f"SCF Energy: {res['scf_energy']}")
    print(f"Optimal Parameters: {res['vqe_result'].optimal_parameters}")
    print(f"Optimizer Evaluations: {res['vqe_result'].optimizer_evals}")
    print(f"Optimizer Time: {res['vqe_result'].optimizer_time}")
    print("\n")


Saving Beryllium.xyz to Beryllium (1).xyz
Loaded molecule:
Be         0.96576        0.06793       -0.06491
converged SCF energy = -14.5033611236956
converged SCF energy = -14.5033611236956
converged SCF energy = -14.5033611236956
converged SCF energy = -14.5033611236956
converged SCF energy = -14.5033611236956
Ansatz: EfficientSU2
VQE Eigenvalue: -14.352279196247823
SCF Energy: -14.503361123695598
Optimal Parameters: {ParameterVectorElement(θ[0]): 3.1406854304600866, ParameterVectorElement(θ[1]): -3.1576062526756328, ParameterVectorElement(θ[2]): -3.1390372793287096, ParameterVectorElement(θ[3]): -0.0006471066395430528, ParameterVectorElement(θ[4]): -3.1410858990454136, ParameterVectorElement(θ[5]): 0.01633928626563257, ParameterVectorElement(θ[6]): 3.144869990239612, ParameterVectorElement(θ[7]): -0.0021735597976207366, ParameterVectorElement(θ[8]): -0.9400532177963796, ParameterVectorElement(θ[9]): -1.1024720080555253, ParameterVectorElement(θ[10]): 4.148683573299722, ParameterVecto

In [ ]:
#Comparison in 3-21g basis

# Upload .xyz file
from google.colab import files
uploaded = files.upload()

# Read the .xyz file and extract atomic coordinates
xyz_file = list(uploaded.keys())[0]
with open(xyz_file, 'r') as f:
    lines = f.readlines()
    atoms = lines[2:]  # Skip the first two lines (number of atoms and comment)
    atom_str = "; ".join(line.strip() for line in atoms)

print(f"Loaded molecule:\n{atom_str}")

# Import necessary libraries
from qiskit.circuit.library import EfficientSU2, RealAmplitudes, TwoLocal, PauliTwoDesign, ExcitationPreserving
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import VQE
from pyscf import gto, scf
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper

# Define function to run VQE with different ansatzes
def run_vqe_with_ansatz(atom_str, ansatz_type):
    # Setup molecule with PySCF
    mol = gto.M(atom=atom_str, basis="321g")
    h_f = scf.RHF(mol).run()

    # Use PySCFDriver to get the Hamiltonian
    driver = PySCFDriver(atom=atom_str)
    problem = driver.run()
    hamiltonian = problem.hamiltonian.second_q_op()

    # Map the Hamiltonian
    mapper = ParityMapper(num_particles=problem.num_particles)
    reduced_op = mapper.map(hamiltonian)

    # Setup ansatz
    if ansatz_type == 'EfficientSU2':
        ansatz = EfficientSU2(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'RealAmplitudes':
        ansatz = RealAmplitudes(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'TwoLocal':
        ansatz = TwoLocal(reduced_op.num_qubits, 'ry', 'cx', reps=1)
    elif ansatz_type == 'PauliTwoDesign':
        ansatz = PauliTwoDesign(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'ExcitationPreserving':
        ansatz = ExcitationPreserving(reduced_op.num_qubits, reps=1)
    else:
        raise ValueError(f"Unknown ansatz type: {ansatz_type}")

    # Setup VQE
    estimator = Estimator()
    optimizer = COBYLA()
    vqe = VQE(estimator, ansatz, optimizer)

    # Compute minimum eigenvalue
    result = vqe.compute_minimum_eigenvalue(reduced_op)

    return result, h_f.e_tot

# Run VQE with different ansatzes
ansatzes = ['EfficientSU2', 'RealAmplitudes', 'TwoLocal', 'PauliTwoDesign', 'ExcitationPreserving']
results = {}
for ansatz in ansatzes:
    vqe_result, scf_energy = run_vqe_with_ansatz(atom_str, ansatz)
    results[ansatz] = {
        'vqe_result': vqe_result,
        'scf_energy': scf_energy
    }

# Compare results
for ansatz, res in results.items():
    print(f"Ansatz: {ansatz}")
    print(f"VQE Eigenvalue: {res['vqe_result'].eigenvalue}")
    print(f"SCF Energy: {res['scf_energy']}")
    print(f"Optimal Parameters: {res['vqe_result'].optimal_parameters}")
    print(f"Optimizer Evaluations: {res['vqe_result'].optimizer_evals}")
    print(f"Optimizer Time: {res['vqe_result'].optimizer_time}")
    print("\n")


Saving Beryllium.xyz to Beryllium (2).xyz
Loaded molecule:
Be         0.96576        0.06793       -0.06491
converged SCF energy = -14.4868202421757
converged SCF energy = -14.4868202421757
converged SCF energy = -14.4868202421757
converged SCF energy = -14.4868202421757
converged SCF energy = -14.4868202421757
Ansatz: EfficientSU2
VQE Eigenvalue: -14.323873890175255
SCF Energy: -14.486820242175671
Optimal Parameters: {ParameterVectorElement(θ[0]): -3.1480656028551866, ParameterVectorElement(θ[1]): -1.975102664396787, ParameterVectorElement(θ[2]): 3.146925083750369, ParameterVectorElement(θ[3]): -3.1437303366103166, ParameterVectorElement(θ[4]): -3.1378691659441977, ParameterVectorElement(θ[5]): -0.7782223443060887, ParameterVectorElement(θ[6]): 3.134449937418046, ParameterVectorElement(θ[7]): 3.139655973510978, ParameterVectorElement(θ[8]): -2.3198651792528944, ParameterVectorElement(θ[9]): 3.1429463156930537, ParameterVectorElement(θ[10]): -2.0944518378670742, ParameterVectorElement(

In [ ]:
#Comparison in 6-31g basis

# Upload .xyz file
from google.colab import files
uploaded = files.upload()

# Read the .xyz file and extract atomic coordinates
xyz_file = list(uploaded.keys())[0]
with open(xyz_file, 'r') as f:
    lines = f.readlines()
    atoms = lines[2:]  # Skip the first two lines (number of atoms and comment)
    atom_str = "; ".join(line.strip() for line in atoms)

print(f"Loaded molecule:\n{atom_str}")

# Import necessary libraries
from qiskit.circuit.library import EfficientSU2, RealAmplitudes, TwoLocal, PauliTwoDesign, ExcitationPreserving
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import VQE
from pyscf import gto, scf
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper

# Define function to run VQE with different ansatzes
def run_vqe_with_ansatz(atom_str, ansatz_type):
    # Setup molecule with PySCF
    mol = gto.M(atom=atom_str, basis="631g")
    h_f = scf.RHF(mol).run()

    # Use PySCFDriver to get the Hamiltonian
    driver = PySCFDriver(atom=atom_str)
    problem = driver.run()
    hamiltonian = problem.hamiltonian.second_q_op()

    # Map the Hamiltonian
    mapper = ParityMapper(num_particles=problem.num_particles)
    reduced_op = mapper.map(hamiltonian)

    # Setup ansatz
    if ansatz_type == 'EfficientSU2':
        ansatz = EfficientSU2(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'RealAmplitudes':
        ansatz = RealAmplitudes(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'TwoLocal':
        ansatz = TwoLocal(reduced_op.num_qubits, 'ry', 'cx', reps=1)
    elif ansatz_type == 'PauliTwoDesign':
        ansatz = PauliTwoDesign(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'ExcitationPreserving':
        ansatz = ExcitationPreserving(reduced_op.num_qubits, reps=1)
    else:
        raise ValueError(f"Unknown ansatz type: {ansatz_type}")

    # Setup VQE
    estimator = Estimator()
    optimizer = COBYLA()
    vqe = VQE(estimator, ansatz, optimizer)

    # Compute minimum eigenvalue
    result = vqe.compute_minimum_eigenvalue(reduced_op)

    return result, h_f.e_tot

# Run VQE with different ansatzes
ansatzes = ['EfficientSU2', 'RealAmplitudes', 'TwoLocal', 'PauliTwoDesign', 'ExcitationPreserving']
results = {}
for ansatz in ansatzes:
    vqe_result, scf_energy = run_vqe_with_ansatz(atom_str, ansatz)
    results[ansatz] = {
        'vqe_result': vqe_result,
        'scf_energy': scf_energy
    }

# Compare results
for ansatz, res in results.items():
    print(f"Ansatz: {ansatz}")
    print(f"VQE Eigenvalue: {res['vqe_result'].eigenvalue}")
    print(f"SCF Energy: {res['scf_energy']}")
    print(f"Optimal Parameters: {res['vqe_result'].optimal_parameters}")
    print(f"Optimizer Evaluations: {res['vqe_result'].optimizer_evals}")
    print(f"Optimizer Time: {res['vqe_result'].optimizer_time}")
    print("\n")


Saving Beryllium.xyz to Beryllium (3).xyz
Loaded molecule:
Be         0.96576        0.06793       -0.06491
converged SCF energy = -14.5667640335096
converged SCF energy = -14.5667640335096
converged SCF energy = -14.5667640335096
converged SCF energy = -14.5667640335096
converged SCF energy = -14.5667640335096
Ansatz: EfficientSU2
VQE Eigenvalue: -14.230766784728802
SCF Energy: -14.56676403350964
Optimal Parameters: {ParameterVectorElement(θ[0]): 0.007315378973930144, ParameterVectorElement(θ[1]): 0.3053619981630669, ParameterVectorElement(θ[2]): -0.012301770868029584, ParameterVectorElement(θ[3]): 0.021570725401537563, ParameterVectorElement(θ[4]): -0.0017265331886193022, ParameterVectorElement(θ[5]): -0.6943877121759731, ParameterVectorElement(θ[6]): 0.8966787617148079, ParameterVectorElement(θ[7]): -2.3467732726099295, ParameterVectorElement(θ[8]): 0.09213575880265165, ParameterVectorElement(θ[9]): 0.23974993773617753, ParameterVectorElement(θ[10]): 3.4156552101421593, ParameterVec

In [ ]:
#Comparison in ccpvdz basis

# Upload .xyz file
from google.colab import files
uploaded = files.upload()

# Read the .xyz file and extract atomic coordinates
xyz_file = list(uploaded.keys())[0]
with open(xyz_file, 'r') as f:
    lines = f.readlines()
    atoms = lines[2:]  # Skip the first two lines (number of atoms and comment)
    atom_str = "; ".join(line.strip() for line in atoms)

print(f"Loaded molecule:\n{atom_str}")

# Import necessary libraries
from qiskit.circuit.library import EfficientSU2, RealAmplitudes, TwoLocal, PauliTwoDesign, ExcitationPreserving
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms import VQE
from pyscf import gto, scf
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper

# Define function to run VQE with different ansatzes
def run_vqe_with_ansatz(atom_str, ansatz_type):
    # Setup molecule with PySCF
    mol = gto.M(atom=atom_str, basis="ccpvdz")
    h_f = scf.RHF(mol).run()

    # Use PySCFDriver to get the Hamiltonian
    driver = PySCFDriver(atom=atom_str)
    problem = driver.run()
    hamiltonian = problem.hamiltonian.second_q_op()

    # Map the Hamiltonian
    mapper = ParityMapper(num_particles=problem.num_particles)
    reduced_op = mapper.map(hamiltonian)

    # Setup ansatz
    if ansatz_type == 'EfficientSU2':
        ansatz = EfficientSU2(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'RealAmplitudes':
        ansatz = RealAmplitudes(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'TwoLocal':
        ansatz = TwoLocal(reduced_op.num_qubits, 'ry', 'cx', reps=1)
    elif ansatz_type == 'PauliTwoDesign':
        ansatz = PauliTwoDesign(reduced_op.num_qubits, reps=1)
    elif ansatz_type == 'ExcitationPreserving':
        ansatz = ExcitationPreserving(reduced_op.num_qubits, reps=1)
    else:
        raise ValueError(f"Unknown ansatz type: {ansatz_type}")

    # Setup VQE
    estimator = Estimator()
    optimizer = COBYLA()
    vqe = VQE(estimator, ansatz, optimizer)

    # Compute minimum eigenvalue
    result = vqe.compute_minimum_eigenvalue(reduced_op)

    return result, h_f.e_tot

# Run VQE with different ansatzes
ansatzes = ['EfficientSU2', 'RealAmplitudes', 'TwoLocal', 'PauliTwoDesign', 'ExcitationPreserving']
results = {}
for ansatz in ansatzes:
    vqe_result, scf_energy = run_vqe_with_ansatz(atom_str, ansatz)
    results[ansatz] = {
        'vqe_result': vqe_result,
        'scf_energy': scf_energy
    }

# Compare results
for ansatz, res in results.items():
    print(f"Ansatz: {ansatz}")
    print(f"VQE Eigenvalue: {res['vqe_result'].eigenvalue}")
    print(f"SCF Energy: {res['scf_energy']}")
    print(f"Optimal Parameters: {res['vqe_result'].optimal_parameters}")
    print(f"Optimizer Evaluations: {res['vqe_result'].optimizer_evals}")
    print(f"Optimizer Time: {res['vqe_result'].optimizer_time}")
    print("\n")


Saving Beryllium.xyz to Beryllium (4).xyz
Loaded molecule:
Be         0.96576        0.06793       -0.06491
converged SCF energy = -14.5723376309534
converged SCF energy = -14.5723376309534
converged SCF energy = -14.5723376309534
converged SCF energy = -14.5723376309534
converged SCF energy = -14.5723376309534
Ansatz: EfficientSU2
VQE Eigenvalue: -14.199119142476125
SCF Energy: -14.572337630953367
Optimal Parameters: {ParameterVectorElement(θ[0]): -3.1396042877058785, ParameterVectorElement(θ[1]): 0.054900731589731404, ParameterVectorElement(θ[2]): -3.1464999790327153, ParameterVectorElement(θ[3]): 3.143708425290163, ParameterVectorElement(θ[4]): 0.003532940427019506, ParameterVectorElement(θ[5]): 3.139035518797738, ParameterVectorElement(θ[6]): -3.1458103293139525, ParameterVectorElement(θ[7]): 1.545628607649241, ParameterVectorElement(θ[8]): 1.2694983771335928, ParameterVectorElement(θ[9]): 0.06929775134563684, ParameterVectorElement(θ[10]): -1.4645499686833365, ParameterVectorEleme

In [ ]:
# Upload .xyz file
from google.colab import files
uploaded = files.upload()

# Read the .xyz file and extract atomic coordinates
xyz_file = list(uploaded.keys())[0]
with open(xyz_file, 'r') as f:
    lines = f.readlines()
    atoms = lines[2:]  # Skip the first two lines (number of atoms and comment)
    atom_str = "; ".join(line.strip() for line in atoms)

print(f"Loaded molecule:\n{atom_str}")

# Import necessary libraries
from qiskit.circuit.library import EfficientSU2, RealAmplitudes, TwoLocal, PauliTwoDesign, ExcitationPreserving
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.optimizers import L_BFGS_B, SPSA, ADAM
from qiskit_algorithms import VQE
from pyscf import gto, scf
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper


# Define function to run VQE with different ansatzes
def run_vqe_with_ansatz(atom_str, ansatz_type, optimizer):
    # Setup molecule with PySCF
    mol = gto.M(atom=atom_str, basis="sto6g")
    h_f = scf.RHF(mol).run()

    # Use PySCFDriver to get the Hamiltonian
    driver = PySCFDriver(atom=atom_str)
    problem = driver.run()
    hamiltonian = problem.hamiltonian.second_q_op()

    # Map the Hamiltonian
    mapper = ParityMapper(num_particles=problem.num_particles)
    reduced_op = mapper.map(hamiltonian)

    # Setup ansatz
    if ansatz_type == 'EfficientSU2':
        ansatz = EfficientSU2(reduced_op.num_qubits, reps=2)  # Increased reps
    elif ansatz_type == 'RealAmplitudes':
        ansatz = RealAmplitudes(reduced_op.num_qubits, reps=2)
    elif ansatz_type == 'TwoLocal':
        ansatz = TwoLocal(reduced_op.num_qubits, 'ry', 'cx', reps=2)
    elif ansatz_type == 'PauliTwoDesign':
        ansatz = PauliTwoDesign(reduced_op.num_qubits, reps=2)
    elif ansatz_type == 'ExcitationPreserving':
        ansatz = ExcitationPreserving(reduced_op.num_qubits, reps=2)
    else:
        raise ValueError(f"Unknown ansatz type: {ansatz_type}")

    # Setup VQE
    estimator = Estimator()
    vqe = VQE(estimator, ansatz, optimizer)

    # Compute minimum eigenvalue
    result = vqe.compute_minimum_eigenvalue(reduced_op)

    return result, h_f.e_tot

# Define optimizers
optimizers = {
    'COBYLA': COBYLA(),
    'L_BFGS_B': L_BFGS_B(),
    'SPSA': SPSA(maxiter=300),
    'ADAM': ADAM()
}

# Run VQE with different ansatzes and optimizers
ansatzes = ['EfficientSU2', 'RealAmplitudes', 'TwoLocal', 'PauliTwoDesign', 'ExcitationPreserving']
results = {}
for ansatz in ansatzes:
    for optimizer_name, optimizer in optimizers.items():
        vqe_result, scf_energy = run_vqe_with_ansatz(atom_str, ansatz, optimizer)
        results[f"{ansatz} with {optimizer_name}"] = {
            'vqe_result': vqe_result,
            'scf_energy': scf_energy
        }

# Compare results
for ansatz_optimizer, res in results.items():
    print(f"Ansatz and Optimizer: {ansatz_optimizer}")
    print(f"VQE Eigenvalue: {res['vqe_result'].eigenvalue}")
    print(f"SCF Energy: {res['scf_energy']}")
    print(f"Optimal Parameters: {res['vqe_result'].optimal_parameters}")
    print(f"Optimizer Evaluations: {res['vqe_result'].optimizer_evals}")
    print(f"Optimizer Time: {res['vqe_result'].optimizer_time}")
    print("\n")


Saving Helium.xyz to Helium (1).xyz
Loaded molecule:
He         0.69054       -0.09173       -0.00471; He         2.38854       -0.09173       -0.00471
converged SCF energy = -5.68478166251705
converged SCF energy = -5.68478166251705
converged SCF energy = -5.68478166251705
converged SCF energy = -5.68478166251705


KeyboardInterrupt: 

In [ ]:
# Install Qiskit and PySCF
!pip install qiskit qiskit-nature pyscf

# Upload your .xyz file
from google.colab import files
uploaded = files.upload()

# Read the .xyz file and extract atomic coordinates
xyz_file = list(uploaded.keys())[0]
with open(xyz_file, 'r') as f:
    lines = f.readlines()
    atoms = lines[2:]  # Skip the first two lines (number of atoms and comment)
    atom_str = "; ".join(line.strip() for line in atoms)

print(f"Loaded molecule:\n{atom_str}")

# Import necessary libraries from PySCF
from pyscf import gto, scf, ci, cc, mp

# List of basis sets to compare
basis_sets = ['sto-3g', 'sto-6g', '3-21g', '6-31g', 'ccpvdz']

# Dictionary to store results
results = {}

for basis in basis_sets:
    # Define molecule with current basis set
    mol = gto.M(
        atom=atom_str,
        basis=basis,
    )

    # Perform HF calculation
    mf = scf.RHF(mol).run()
    hf_energy = mf.e_tot

    # Perform post-HF calculations
    mp2_energy = mp.MP2(mf).run().e_tot
    cisd_energy = ci.CISD(mf).run().e_tot
    ccsd_energy = cc.CCSD(mf).run().e_tot

    # Store results
    results[basis] = {
        'HF Energy': hf_energy,
        'MP2 Energy': mp2_energy,
        'CISD Energy': cisd_energy,
        'CCSD Energy': ccsd_energy,
    }

# Print results
for basis, energies in results.items():
    print(f"Basis Set: {basis}")
    for method, energy in energies.items():
        print(f"{method}: {energy}")
    print("\n")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 13.4 MB/s eta 0:00:00


Saving Helium.xyz to Helium.xyz
Loaded molecule:
He         0.69054       -0.09173       -0.00471; He         2.38854       -0.09173       -0.00471
converged SCF energy = -5.60785080373275
E(MP2) = -5.60785080373275  E_corr = 0
E(SCS-MP2) = -5.60785080373275  E_corr = 0
E(RCISD) = -5.607850803732752  E_corr = 0
E(CCSD) = -5.607850803732752  E_corr = 0
converged SCF energy = -5.68478166251705
E(MP2) = -5.68478166251705  E_corr = 0
E(SCS-MP2) = -5.68478166251705  E_corr = 0
E(RCISD) = -5.68478166251705  E_corr = 0
E(CCSD) = -5.68478166251705  E_corr = 0
converged SCF energy = -5.66395524575708
E(MP2) = -5.68683723457669  E_corr = -0.0228819888196064
E(SCS-MP2) = -5.6914134743035  E_corr = -0.0274582285464158
E(RCISD) = -5.693480358348403  E_corr = -0.02952511259132204
E(CCSD) = -5.69357161113531  E_corr = -0.02961636537822869
converged SCF energy = -5.70214626642299
E(MP2) = -5.72431271323143  E_corr = -0.0221664468084408
E(SCS-MP2) = -5.72874555724735  E_corr = -0.0265992908243546
E(RCI

In [ ]:
!pip install qiskit[visualization]==1.0.2
!pip install qiskit_aer
!pip install qiskit_ibm_runtime
!pip install matplotlib
!pip install pylatexenc
!pip install pyscf

import numpy as np
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Pauli
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit.circuit.library import QFT, PauliEvolutionGate
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
from qiskit.primitives import StatevectorSampler, PrimitiveJob
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

def get_qubit_op(dist):
    # Define Molecule
    molecule = MoleculeInfo(
        # Coordinates in Angstrom
        symbols=["Li", "H"],
        coords=([0.0, 0.0, 0.0], [dist, 0.0, 0.0]),
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)
    properties = driver.run()

    # Reduce number of qubits used -> freeze core & remove 2 unoccupied orbitals
    problem = FreezeCoreTransformer(
        freeze_core=True, remove_orbitals=[-3, -2]
    ).transform(properties)

    num_particles = problem.num_particles
    num_spatial_orbitals = problem.num_spatial_orbitals

    mapper = ParityMapper(num_particles=num_particles)
    qubit_op = mapper.map(problem.second_q_ops()[0])
    return qubit_op, num_particles, num_spatial_orbitals, problem, mapper

from qiskit_aer.primitives import Estimator

#exact GSE is calculated on qubit operator and classical exact eigensolver.
#exact eigensolver (full config interaction (fci)): calculates exact eigenvalues and eigenvectos
def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result   # initial state |psi> is created -> VQE uses to produce final ansatz = min_theta (|psi(theta)>)

noiseless_estimator = Estimator(approximation=True)

dist=1.6
(qubit_op, num_particles, num_spatial_orbitals, problem, mapper) = get_qubit_op(dist)

# Calculate exact eigenvalues with NumPyEigensolver for reference
result = exact_solver(qubit_op, problem)
exact_energy = result.total_energies[0].real
print(f"Exact ground state energy: {exact_energy} Hartree")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136816 sha256=5acc0de803d78403463f9142654943b45510c9a57ee7e01b6d2d4224dfc67e0d
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc
  Attempting uninstall: qiskit
    Found existing installation: qiskit 1.1.1
    Uninstalling qiskit-1.1.1:
      Successfully uninstalled qiskit-1.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 24.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadat